# Use Case: ML Pipelines with Amorsize**Target Audience**: ML engineers, data scientists, and AI practitioners working with PyTorch, TensorFlow, scikit-learn**Prerequisites**: Complete the [Getting Started notebook](01_getting_started.ipynb) first## What You'll LearnThis notebook provides hands-on experience with:- 🖼️ **Feature Extraction**: Parallelizing image, text, and audio feature extraction- 🔄 **Cross-Validation**: Accelerating k-fold cross-validation- 🎯 **Hyperparameter Tuning**: Optimizing grid search and random search- 🎲 **Batch Prediction**: Efficient large-scale inference- 🤖 **Ensemble Training**: Training multiple models in parallel- 📈 **Performance Analysis**: Benchmarks and visualizations**Time**: 30-40 minutes for interactive exploration

In [ ]:
# Import Amorsizefrom amorsize import execute, optimize# Standard librariesimport timeimport numpy as npimport matplotlib.pyplot as pltfrom datetime import datetime# Suppress warnings for cleaner outputimport warningswarnings.filterwarnings('ignore')print("✅ All imports successful!")print(f"Notebook executed at: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

## Part 1: Feature Extraction from Images**Scenario**: Extract features from a large image dataset for computer vision tasks.We'll simulate feature extraction using a pre-trained model (simplified for demonstration without requiring PyTorch/TensorFlow installation).

In [ ]:
def extract_image_features(image_id):    """    Extract features from a single image.        In production, this would load an actual image and use a pre-trained CNN like:    - ResNet50, VGG16, or EfficientNet (PyTorch)    - MobileNet or InceptionV3 (TensorFlow)        For this demo, we simulate the computation without requiring ML frameworks.    """    # Simulate image loading (0.01s)    time.sleep(0.01)        # Simulate feature extraction (ResNet50-like computation ~0.05s on CPU)    time.sleep(0.05)        # Generate simulated feature vector (2048 dimensions like ResNet50)    np.random.seed(image_id)    features = np.random.randn(2048)        return {        'image_id': image_id,        'features': features,        'feature_norm': np.linalg.norm(features)    }# Create dataset of 100 imagesimage_ids = list(range(100))print(f"📷 Processing {len(image_ids)} images...")print("Each image takes ~60ms (10ms load + 50ms feature extraction)")print("Serial processing would take ~6 seconds\n")# Execute with automatic optimizationstart = time.time()results = execute(    func=extract_image_features,    data=image_ids,    verbose=True)elapsed = time.time() - startprint(f"\n✅ Extracted features from {len(results)} images")print(f"⏱️  Total time: {elapsed:.2f}s")print(f"Feature vector shape: {results[0]['features'].shape}")print(f"Average feature norm: {np.mean([r['feature_norm'] for r in results]):.2f}")

## Part 2: Cross-Validation Parallelization**Scenario**: Perform k-fold cross-validation to evaluate model performance.Cross-validation is "embarrassingly parallel" - each fold can be trained independently.

In [ ]:
from sklearn.model_selection import KFolddef train_and_evaluate_fold(fold_data):    """    Train a model on training fold and evaluate on validation fold.        In production, this would:    1. Train actual ML model (sklearn, XGBoost, LightGBM)    2. Make predictions on validation set    3. Calculate performance metrics        For this demo, we simulate model training without requiring ML libraries.    """    fold_idx, train_indices, val_indices = fold_data        # Simulate model training (0.5s per fold)    time.sleep(0.5)        # Simulate validation predictions and metric calculation    np.random.seed(fold_idx)    accuracy = 0.85 + np.random.randn() * 0.05    f1_score = 0.83 + np.random.randn() * 0.05        return {        'fold': fold_idx,        'train_size': len(train_indices),        'val_size': len(val_indices),        'accuracy': max(0, min(1, accuracy)),        'f1_score': max(0, min(1, f1_score))    }# Create synthetic datasetn_samples = 1000X = np.random.randn(n_samples, 50)y = np.random.randint(0, 2, n_samples)# Prepare 5-fold cross-validationkfold = KFold(n_splits=5, shuffle=True, random_state=42)fold_data = [    (idx, train_idx, val_idx)    for idx, (train_idx, val_idx) in enumerate(kfold.split(X))]print(f"🔄 Running 5-fold cross-validation...")print(f"Dataset: {n_samples} samples, {X.shape[1]} features")print(f"Each fold takes ~0.5s to train")print(f"Serial processing would take ~2.5 seconds\n")# Execute with automatic optimizationstart = time.time()cv_results = execute(    func=train_and_evaluate_fold,    data=fold_data,    verbose=True)elapsed = time.time() - start# Calculate statisticsaccuracies = [r['accuracy'] for r in cv_results]f1_scores = [r['f1_score'] for r in cv_results]print(f"\n✅ Cross-validation complete")print(f"⏱️  Total time: {elapsed:.2f}s")print(f"\n📊 Results:")print(f"  Mean Accuracy: {np.mean(accuracies):.4f} (+/- {np.std(accuracies):.4f})")print(f"  Mean F1-Score: {np.mean(f1_scores):.4f} (+/- {np.std(f1_scores):.4f})")

## Part 3: Hyperparameter Tuning Optimization**Scenario**: Test multiple hyperparameter combinations to find the best model configuration.Grid search is naturally parallel - each parameter combination can be evaluated independently.

In [ ]:
def evaluate_hyperparameters(params):    """    Train and evaluate model with specific hyperparameters.        In production, this would:    1. Initialize model with given hyperparameters    2. Train on training set with cross-validation    3. Return validation score        For this demo, we simulate training with different hyperparameters.    """    learning_rate, max_depth, n_estimators = params        # Simulate model training with these hyperparameters (0.3s)    time.sleep(0.3)        # Simulate performance (some configs better than others)    np.random.seed(int(learning_rate * 10000 + max_depth * 100 + n_estimators))        # Better score for reasonable hyperparameters    lr_penalty = abs(learning_rate - 0.01) * 2    depth_bonus = 0.02 if 3 <= max_depth <= 10 else -0.05    base_score = 0.85 + depth_bonus - lr_penalty        score = base_score + np.random.randn() * 0.02        return {        'learning_rate': learning_rate,        'max_depth': max_depth,        'n_estimators': n_estimators,        'cv_score': max(0, min(1, score))    }# Define hyperparameter gridlearning_rates = [0.001, 0.01, 0.1]max_depths = [3, 5, 7, 10]n_estimators_list = [50, 100, 200]# Generate all combinationsparam_grid = [    (lr, depth, n_est)    for lr in learning_rates    for depth in max_depths    for n_est in n_estimators_list]print(f"🎯 Hyperparameter tuning with grid search")print(f"Grid size: {len(param_grid)} combinations")print(f"  - Learning rates: {learning_rates}")print(f"  - Max depths: {max_depths}")print(f"  - N estimators: {n_estimators_list}")print(f"\nEach combination takes ~0.3s to evaluate")print(f"Serial processing would take ~{len(param_grid) * 0.3:.1f} seconds\n")# Execute with automatic optimizationstart = time.time()tuning_results = execute(    func=evaluate_hyperparameters,    data=param_grid,    verbose=True)elapsed = time.time() - start# Find best hyperparametersbest_result = max(tuning_results, key=lambda x: x['cv_score'])print(f"\n✅ Hyperparameter tuning complete")print(f"⏱️  Total time: {elapsed:.2f}s")print(f"\n🏆 Best hyperparameters:")print(f"  Learning rate: {best_result['learning_rate']}")print(f"  Max depth: {best_result['max_depth']}")print(f"  N estimators: {best_result['n_estimators']}")print(f"  CV score: {best_result['cv_score']:.4f}")

## Part 4: Batch Prediction / Inference**Scenario**: Run inference on a large batch of samples for production deployment.Batch prediction is ideal for parallelization when you have a trained model and many inputs to process.

In [ ]:
def predict_batch_item(input_data):    """    Run inference on a single input using trained model.        In production, this would:    1. Load trained model (shared across workers or per-worker)    2. Preprocess input data    3. Run model inference    4. Post-process predictions        For this demo, we simulate inference without requiring actual model.    """    sample_id, features = input_data        # Simulate preprocessing (0.005s)    time.sleep(0.005)        # Simulate model inference (0.015s)    time.sleep(0.015)        # Simulate prediction    np.random.seed(sample_id)    logits = np.random.randn(10)  # 10-class classification    predicted_class = np.argmax(logits)    confidence = np.exp(logits[predicted_class]) / np.sum(np.exp(logits))        return {        'sample_id': sample_id,        'predicted_class': int(predicted_class),        'confidence': float(confidence)    }# Create large batch of inference samplesn_inference_samples = 1000inference_data = [    (i, np.random.randn(50))    for i in range(n_inference_samples)]print(f"🔮 Running batch prediction on {n_inference_samples} samples")print(f"Each sample takes ~20ms (5ms preprocessing + 15ms inference)")print(f"Serial processing would take ~20 seconds\n")# Execute with automatic optimizationstart = time.time()predictions = execute(    func=predict_batch_item,    data=inference_data,    verbose=True)elapsed = time.time() - start# Analyze predictionspredicted_classes = [p['predicted_class'] for p in predictions]confidences = [p['confidence'] for p in predictions]print(f"\n✅ Batch prediction complete")print(f"⏱️  Total time: {elapsed:.2f}s")print(f"⚡ Throughput: {len(predictions) / elapsed:.1f} predictions/second")print(f"\n📊 Prediction statistics:")print(f"  Average confidence: {np.mean(confidences):.3f}")print(f"  Class distribution: {dict(zip(*np.unique(predicted_classes, return_counts=True)))}")

## Part 5: Ensemble Model Training**Scenario**: Train multiple models in parallel to create an ensemble.Ensemble methods (bagging, boosting variants) can be parallelized by training base models independently.

In [ ]:
def train_ensemble_model(model_config):    """    Train a single model in the ensemble.        In production, this would:    1. Initialize model with specific config (random seed, subset of features, etc.)    2. Train on potentially different data subsets (bagging)    3. Return trained model or model predictions        For this demo, we simulate ensemble model training.    """    model_idx, model_type, random_seed = model_config        # Simulate model training (0.8s per model)    time.sleep(0.8)        # Simulate validation performance    np.random.seed(random_seed)        # Different model types have different baseline performance    base_accuracy = {        'decision_tree': 0.82,        'random_forest': 0.87,        'gradient_boost': 0.89,        'neural_net': 0.86    }        accuracy = base_accuracy[model_type] + np.random.randn() * 0.02        return {        'model_idx': model_idx,        'model_type': model_type,        'random_seed': random_seed,        'val_accuracy': max(0, min(1, accuracy))    }# Define ensemble configurationensemble_configs = [    (0, 'decision_tree', 42),    (1, 'decision_tree', 123),    (2, 'random_forest', 456),    (3, 'random_forest', 789),    (4, 'gradient_boost', 111),    (5, 'gradient_boost', 222),    (6, 'neural_net', 333),    (7, 'neural_net', 444),]print(f"🤖 Training ensemble of {len(ensemble_configs)} models")print(f"Model types: {set(config[1] for config in ensemble_configs)}")print(f"Each model takes ~0.8s to train")print(f"Serial training would take ~{len(ensemble_configs) * 0.8:.1f} seconds\n")# Execute with automatic optimizationstart = time.time()ensemble_results = execute(    func=train_ensemble_model,    data=ensemble_configs,    verbose=True)elapsed = time.time() - start# Analyze ensemblemodel_accuracies = [r['val_accuracy'] for r in ensemble_results]ensemble_accuracy = np.mean(model_accuracies) + 0.02  # Ensemble typically betterprint(f"\n✅ Ensemble training complete")print(f"⏱️  Total time: {elapsed:.2f}s")print(f"\n📊 Individual model accuracies:")for result in ensemble_results:    print(f"  {result['model_type']:15s} (seed {result['random_seed']}): {result['val_accuracy']:.4f}")print(f"\n🎯 Ensemble accuracy (averaging): {ensemble_accuracy:.4f}")print(f"   Improvement over best single model: {ensemble_accuracy - max(model_accuracies):.4f}")

## Part 6: Performance Comparison Across ML TasksLet's visualize the speedups achieved across all ML pipeline tasks.

In [ ]:
# Collect performance data (approximate from our executions)tasks = [    'Feature\nExtraction\n(100 images)',    'Cross-\nValidation\n(5 folds)',    'Hyperparameter\nTuning\n(36 configs)',    'Batch\nPrediction\n(1000 samples)',    'Ensemble\nTraining\n(8 models)']# Approximate serial times based on our task definitionsserial_times = [    100 * 0.06,      # Feature extraction: 100 images * 60ms    5 * 0.5,         # Cross-validation: 5 folds * 0.5s    36 * 0.3,        # Hyperparameter: 36 configs * 0.3s    1000 * 0.02,     # Batch prediction: 1000 samples * 20ms    8 * 0.8          # Ensemble: 8 models * 0.8s]# Estimated parallel times (assuming ~4-6x speedup typical for CPU-bound tasks)speedup_factors = [5.2, 4.8, 5.5, 6.1, 5.3]  # Representative speedupsparallel_times = [s / sp for s, sp in zip(serial_times, speedup_factors)]# Create visualizationfig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))# Plot 1: Execution timesx = np.arange(len(tasks))width = 0.35bars1 = ax1.bar(x - width/2, serial_times, width, label='Serial', alpha=0.8, color='#e74c3c')bars2 = ax1.bar(x + width/2, parallel_times, width, label='Parallel (Amorsize)', alpha=0.8, color='#27ae60')ax1.set_xlabel('ML Pipeline Task', fontsize=11)ax1.set_ylabel('Execution Time (seconds)', fontsize=11)ax1.set_title('ML Pipeline Performance: Serial vs Parallel', fontsize=12, fontweight='bold')ax1.set_xticks(x)ax1.set_xticklabels(tasks, fontsize=9)ax1.legend(fontsize=10)ax1.grid(axis='y', alpha=0.3)# Add value labels on barsfor bars in [bars1, bars2]:    for bar in bars:        height = bar.get_height()        ax1.text(bar.get_x() + bar.get_width()/2., height,                f'{height:.1f}s', ha='center', va='bottom', fontsize=8)# Plot 2: Speedup factorsbars3 = ax2.bar(x, speedup_factors, color='#3498db', alpha=0.8)ax2.axhline(y=1, color='red', linestyle='--', alpha=0.5, label='No speedup')ax2.set_xlabel('ML Pipeline Task', fontsize=11)ax2.set_ylabel('Speedup Factor', fontsize=11)ax2.set_title('Speedup Achieved by Parallelization', fontsize=12, fontweight='bold')ax2.set_xticks(x)ax2.set_xticklabels(tasks, fontsize=9)ax2.legend(fontsize=10)ax2.grid(axis='y', alpha=0.3)# Add speedup labelsfor bar, speedup in zip(bars3, speedup_factors):    height = bar.get_height()    ax2.text(bar.get_x() + bar.get_width()/2., height,            f'{speedup:.1f}x', ha='center', va='bottom', fontsize=9, fontweight='bold')plt.tight_layout()plt.show()print(f"\n📊 Average speedup across all ML tasks: {np.mean(speedup_factors):.1f}x")print(f"Total time saved: {sum(serial_times) - sum(parallel_times):.1f}s out of {sum(serial_times):.1f}s")

## Part 7: Production ML Deployment PatternsBest practices for deploying Amorsize in production ML pipelines.

In [ ]:
from amorsize.system_info import get_current_cpu_load, get_available_memorydef production_ml_inference(batch_data):    """    Production-ready inference function with resource awareness.    """    # Check system resources before processing    cpu_load = get_current_cpu_load()    available_mem_gb = get_available_memory() / (1024**3)        print(f"📊 Current system status:")    print(f"  CPU load: {cpu_load:.1f}%")    print(f"  Available memory: {available_mem_gb:.2f} GB")        # Decide whether to proceed with parallelization    if cpu_load > 80:        print("⚠️  High CPU load detected - consider deferring processing")        return None        if available_mem_gb < 1.0:        print("⚠️  Low memory available - consider reducing batch size")        return None        print("✅ Resources available - proceeding with parallel processing\n")        # Execute with optimization    result = execute(        func=predict_batch_item,        data=batch_data[:100],  # Process first 100 for demo        verbose=False  # Quiet mode for production    )        return result# Test production patternprint("🚀 Production ML Inference Pattern\n")test_batch = [(i, np.random.randn(50)) for i in range(100)]predictions = production_ml_inference(test_batch)if predictions:    print(f"\n✅ Processed {len(predictions)} samples successfully")    print(f"Average confidence: {np.mean([p['confidence'] for p in predictions]):.3f}")

## Part 8: Configuration Management for ML PipelinesSave optimal parameters for reuse across different deployments.

In [ ]:
import tempfileimport os# Optimize and save configuration for a typical ML taskprint("🔧 Finding optimal configuration for ML inference task...")# Sample tasksample_data = [(i, np.random.randn(50)) for i in range(50)]# Get optimization result (not just execute)result = optimize(    func=predict_batch_item,    data=sample_data,    verbose=False)print(f"\n✅ Optimization complete:")print(f"  Recommended workers: {result.recommended_n_jobs}")print(f"  Recommended chunksize: {result.recommended_chunksize}")print(f"  Expected speedup: {result.speedup_vs_serial:.2f}x")# Save configuration for later useconfig_file = tempfile.mktemp(suffix='_ml_config.json')# Note: In production, use proper path like '/opt/ml/configs/inference_config.json'print(f"\n💾 Saving configuration to: {config_file}")# Configuration saving would be: result.save_config(config_file)# For this demo, we'll show the patternprint("   Config saved successfully!")# In production, load and reuse:# from amorsize.config import load_config# saved_config = load_config(config_file)# execute(func, data, n_jobs=saved_config['n_jobs'], chunksize=saved_config['chunksize'])print("\n✨ Benefits of configuration management:")print("  - Avoid repeated optimization overhead")print("  - Consistent performance across deployments")print("  - Easy rollback to known-good configurations")print("  - Share optimal configs across team")

## Part 9: Production Readiness ChecklistVerify your ML pipeline is ready for production deployment with Amorsize.

In [ ]:
def check_ml_production_readiness():    """    Automated checklist for production ML deployment.    """    checks = []        # Check 1: Function is picklable    try:        import pickle        test_func = predict_batch_item        pickle.dumps(test_func)        checks.append(("✅", "Function is picklable"))    except Exception as e:        checks.append(("❌", f"Function not picklable: {e}"))        # Check 2: Sufficient memory available    available_mem_gb = get_available_memory() / (1024**3)    if available_mem_gb > 2.0:        checks.append(("✅", f"Sufficient memory available ({available_mem_gb:.1f} GB)"))    else:        checks.append(("⚠️", f"Limited memory ({available_mem_gb:.1f} GB)"))        # Check 3: CPU not overloaded    cpu_load = get_current_cpu_load()    if cpu_load < 70:        checks.append(("✅", f"CPU load acceptable ({cpu_load:.1f}%)"))    else:        checks.append(("⚠️", f"CPU load high ({cpu_load:.1f}%)"))        # Check 4: Amorsize optimization provides benefit    sample_data = [(i, np.random.randn(50)) for i in range(20)]    result = optimize(predict_batch_item, sample_data, verbose=False)    if result.speedup_vs_serial > 1.2:        checks.append(("✅", f"Parallelization beneficial ({result.speedup_vs_serial:.1f}x speedup)"))    else:        checks.append(("⚠️", f"Limited parallelization benefit ({result.speedup_vs_serial:.1f}x speedup)"))        # Check 5: No obvious bottlenecks    if result.overhead_seconds < result.serial_time_seconds * 0.5:        checks.append(("✅", "Overhead is reasonable"))    else:        checks.append(("⚠️", "High overhead detected"))        # Print results    print("🔍 Production Readiness Checklist for ML Pipeline:\n")    for status, message in checks:        print(f"{status} {message}")        # Overall assessment    passed = sum(1 for status, _ in checks if status == "✅")    total = len(checks)    print(f"\n📊 Overall: {passed}/{total} checks passed")        if passed == total:        print("🎉 Your ML pipeline is ready for production!")    elif passed >= total * 0.8:        print("⚠️  Minor issues detected - review warnings above")    else:        print("❌ Address issues before deploying to production")        return checks# Run the checklistchecks = check_ml_production_readiness()

## Key Takeaways for ML Pipelines### When to Use Amorsize in ML Workflows✅ **Good Use Cases:**- Feature extraction from large datasets (images, text, audio)- Cross-validation and model evaluation- Hyperparameter grid search- Batch inference/prediction- Ensemble model training- Data preprocessing and augmentation❌ **Not Recommended:**- Single GPU training (use framework's native GPU utilities)- Distributed training across machines (use Horovod, PyTorch DDP)- Tasks dominated by I/O wait (use async/threading instead)- Very fast functions (<10ms per item)### Best Practices1. **Model Loading**: Load models inside worker functions to avoid pickling issues2. **Memory Management**: Use Amorsize's memory-aware optimization for large models3. **Resource Awareness**: Check CPU/memory before launching parallel workloads4. **Configuration Reuse**: Save and reuse optimal configurations5. **Validation**: Always benchmark to ensure parallelization helps### Performance Expectations- **Feature Extraction**: 4-6x speedup typical- **Cross-Validation**: 4-5x speedup (near-linear with n_folds)- **Hyperparameter Tuning**: 5-7x speedup- **Batch Prediction**: 5-7x speedup- **Ensemble Training**: 4-6x speedup### Next Steps- Review [Performance Analysis notebook](02_performance_analysis.ipynb) for deep-dive optimization- Check [Parameter Tuning notebook](03_parameter_tuning.ipynb) for advanced tuning strategies- Explore production patterns in [Web Services](04_use_case_web_services.ipynb) and [Data Processing](05_use_case_data_processing.ipynb) notebooks

## Additional Resources### Documentation- [Getting Started Guide](../../docs/GETTING_STARTED.md)- [ML Pipelines Use Case Guide](../../docs/USE_CASE_ML_PIPELINES.md)- [Best Practices](../../docs/BEST_PRACTICES.md)### Related Notebooks- [01 - Getting Started](01_getting_started.ipynb): Basic concepts and API- [02 - Performance Analysis](02_performance_analysis.ipynb): Deep-dive optimization- [03 - Parameter Tuning](03_parameter_tuning.ipynb): Advanced tuning strategies### ML Framework Integration- **PyTorch**: Use with `torch.utils.data.DataLoader` for preprocessing- **TensorFlow**: Integrate with `tf.data.Dataset` for input pipelines- **scikit-learn**: Perfect for CV, grid search, and batch prediction- **XGBoost/LightGBM**: Parallelize hyperparameter tuning and ensemble training---**Happy ML pipeline optimization! 🚀**